In [1]:
import pandas as pd
import numpy as np

In [2]:
# %load ../../../helpers.py
import datetime
import json

def dated_filename (fn, ext='.csv'):
    today = datetime.date.today()
    return '{}-{}{}'.format(fn, today, ext)

def load_json (fn):
    with open(fn, 'r') as myfile:
        data=myfile.read()
    return json.loads(data)


In [3]:
users = pd.read_csv(
    '../../data/IT.NET.USER.ZS/API_IT.NET.USER.ZS_DS2_en_csv_v2_10576990.csv',
)
users.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Aruba,ABW,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,62.0,69.0,74.000000,78.9,83.78,88.661227,93.542454,97.170000,NaN,NaN
1,Afghanistan,AFG,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,5.0,5.454545,5.9,7.00,8.260000,10.595726,11.447688,NaN,NaN
2,Angola,AGO,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,2.8,3.1,6.500000,8.9,10.20,12.400000,13.000000,14.339079,NaN,NaN
3,Albania,ALB,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,45.0,49.0,54.655959,57.2,60.10,63.252933,66.363445,71.847041,NaN,NaN
4,Andorra,AND,Individuals using the Internet (% of population),IT.NET.USER.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,81.0,81.0,86.434425,94.0,95.90,96.910000,97.930637,98.871436,NaN,NaN


In [4]:
def latest_statistic (row):
    latest = row.dropna()
    try:
        return float(latest[-1])
    except:
        return np.nan

users['latest'] = users.apply(latest_statistic, axis=1).apply(lambda x: x/100)

In [63]:
users['latest']

0      0.971700
1      0.114477
2      0.143391
3      0.718470
4      0.988714
5      0.488476
6      0.948199
7      0.758097
8      0.697181
9      0.000000
10     0.760000
11     0.865447
12     0.879356
13     0.790000
14     0.055947
15     0.876797
16     0.141190
17     0.158762
18     0.180200
19     0.634101
20     0.958781
21     0.850000
22     0.694912
23     0.744364
24     0.470826
25     0.983700
26     0.438323
27     0.674700
28     0.817608
29     0.948667
         ...   
234    0.622891
235    0.274927
236    0.498162
237    0.412487
238    0.301878
239    0.221182
240    0.773261
241    0.555000
242    0.646846
243    0.493183
244    0.160000
245    0.237065
246    0.571151
247    0.583287
248    0.682797
249    0.752326
250    0.523051
251    0.655600
252    0.643134
253    0.376000
254    0.643775
255    0.495691
256    0.257198
257    0.485651
258    0.336109
259         NaN
260    0.267184
261    0.561674
262    0.278526
263    0.270555
Name: latest, Length: 26

In [5]:
pops = pd.read_csv(
    '../../data/SP.POP.TOTL/API_SP.POP.TOTL_DS2_en_csv_v2_10576638.csv'
)

pops.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,Unnamed: 63
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0,NaN,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0,NaN,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,29784193.0,NaN,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,NaN,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,76965.0,NaN,NaN


In [6]:
def latest_population (row):
    latest = row.dropna()
    return pd.to_numeric(latest[-1], errors='coerce') # convert non-numbers to NaN

pops['latest'] = pops.apply(latest_population, axis=1).dropna()
pops['latest']

0      1.052640e+05
1      3.553008e+07
2      2.978419e+07
3      2.873457e+06
4      7.696500e+04
5      4.144919e+08
6      9.400145e+06
7      4.427104e+07
8      2.930450e+06
9      5.564100e+04
10     1.020120e+05
11     2.460186e+07
12     8.797566e+06
13     9.854033e+06
14     1.086424e+07
15     1.138239e+07
16     1.117569e+07
17     1.919338e+07
18     1.646698e+08
19     7.075947e+06
20     1.492584e+06
21     3.953610e+05
22     3.507017e+06
23     9.498264e+06
24     3.746810e+05
25     6.544100e+04
26     1.105160e+07
27     2.092883e+08
28     2.857190e+05
29     4.286970e+05
           ...     
234    6.280598e+08
235    1.296311e+06
236    3.752170e+08
237    1.080200e+05
238    1.788389e+09
239    1.061108e+09
240    1.369125e+06
241    1.153213e+07
242    8.074502e+07
243    1.119200e+04
244    5.731002e+07
245    4.286296e+07
246    4.483114e+07
247    2.576181e+09
248    3.456750e+06
249    3.251471e+08
250    3.238720e+07
251    1.098970e+05
252    3.197706e+07


In [99]:

merged = users.merge(pops, how='right', on='Country Code')
merged = merged.rename(columns={
    'latest_x': 'percent-online', 
    'latest_y': 'population',
    'Country Name_x': 'Country Name',
})
merged['people-online'] = merged.apply(lambda row: row['population']*row['percent-online'], axis=1)

In [100]:
out = merged[['Country Code', 'Country Name', 'people-online']]

In [101]:
out.to_csv(dated_filename('internet-users-country'))

In [103]:
out

,Country Code,Country Name,people-online
0,ABW,Aruba,1.022850e+05
1,AFG,Afghanistan,4.067373e+06
2,AGO,Angola,4.270779e+06
3,ALB,Albania,2.064494e+06
4,AND,Andorra,7.609640e+04
5,ARB,Arab World,2.024694e+08
6,ARE,United Arab Emirates,8.913210e+06
7,ARG,Argentina,3.356176e+07
8,ARM,Armenia,2.043055e+06
9,ASM,American Samoa,0.000000e+00
